# Automated Sale Notifier
### Notifies you when the sale is up and running!

*This particular program sends you a notification whenever the specified product is at your desired price.*

*I have used simple web scraping technique for the same, where it keeps a record of the price at every hour, and notifies through mail when the price drops below a certain value.*

In [1]:
# importing  libraries 

from bs4 import BeautifulSoup   #python library used for pulling data out of HTML files
import requests                 #library for making http requests
import time                     #python time module provides functions for working with times
import datetime                 #datetime module helps to work with both date and time
import smtplib                  #this module can be used to send an email to any internet machine

In [2]:
# Connecting to the webpage

url = 'https://www.amazon.in/New-Apple-Watch-GPS-44mm/dp/B08J6HRBLH?ref_=Oct_DLandingS_D_8dc5a645_60&smid=A14CZOWI0VEHLG'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

page = requests.get(url, headers=headers)

page_content = BeautifulSoup(page.content, "html.parser") # returns the html code for the entire page

# Finding the Product name,price and availability

product = page_content.find(id='productTitle').get_text()
price = page_content.find(id='priceblock_dealprice').get_text()
stock = page_content.find(id= 'availability').get_text()

print(product)
print(price)
print(stock)









New Apple Watch SE (GPS, 44mm) - Space Grey Aluminium Case with Black Sport Band







₹27,900.00




In stock.







*We are getting so many white spaces because we are pulling out the information from the html code, which is not much organized.*
*Therfore we would now remove the whitespaces.*

In [3]:
price = price.strip()[1:]
product = product.strip()
stock = stock.strip()
print(product)
print(price)
print(stock)

New Apple Watch SE (GPS, 44mm) - Space Grey Aluminium Case with Black Sport Band
27,900.00
In stock.


*It is always helpful to have a timestamp column in a dataset, so therefore we will also keep a track of the date and time at which the data was recorded*

In [4]:
now = datetime.datetime.now()
date = datetime.date.today()
t = now.strftime("%X")

print(date)
print(t)

2021-10-27
23:35:22


*We would now create a csv file to store the data.*

In [5]:
import csv

column_header = ['Product', 'Price', 'Availability' , 'Date' , 'Time']
values = [product, price, stock, date, t]


with open('WebScrapingData.csv', 'w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(column_header)
    writer.writerow(values)

*The following function keeps on recording the values into the csv file, we have combined all the above statements and are appending the csv file at the end, And this will also call another function in case the price drops.*

In [6]:
def pricecheck():
    url = 'https://www.amazon.in/New-Apple-Watch-GPS-44mm/dp/B08J6HRBLH?ref_=Oct_DLandingS_D_8dc5a645_60&smid=A14CZOWI0VEHLG'
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    page = requests.get(url, headers=headers)
    page_content = BeautifulSoup(page.content, "html.parser")
    product = page_content.find(id='productTitle').get_text()
    price = page_content.find(id='priceblock_dealprice').get_text()
    stock = page_content.find(id= 'availability').get_text()
    price = price.strip()[1:]
    product = product.strip()
    stock = stock.strip()
    now = datetime.datetime.now()
    date = datetime.date.today()
    t = now.strftime("%X")
    values = [product, price, stock, date, t]
    with open('WebScrapingData.csv', 'a+', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(values)
    if(price == "25,900"):
        e_mail()

*We are now checking the values at regular intervals, in this case in every 10 seconds.*

In [7]:
while(True):
    pricecheck()
    time.sleep(10)

KeyboardInterrupt: 

*We can convert the csv into a dataframe and perform more functions if we want.*

In [8]:
import pandas as pd

webscraping_df = pd.read_csv(r'C:\Users\Rohan\Desktop\WebScrapingData.csv')

webscraping_df

,Product,Price,Availability,Date,Time
0,"New Apple Watch SE (GPS, 44mm) - Space Grey Al...","27,900.00",In stock.,2021-10-27,23:35:22
1,"New Apple Watch SE (GPS, 44mm) - Space Grey Al...","27,900.00",In stock.,2021-10-27,23:35:55
2,"New Apple Watch SE (GPS, 44mm) - Space Grey Al...","27,900.00",In stock.,2021-10-27,23:36:07
3,"New Apple Watch SE (GPS, 44mm) - Space Grey Al...","27,900.00",In stock.,2021-10-27,23:36:19
4,"New Apple Watch SE (GPS, 44mm) - Space Grey Al...","27,900.00",In stock.,2021-10-27,23:36:31
...,...,...,...,...,...
535,"New Apple Watch SE (GPS, 44mm) - Space Grey Al...","27,900.00",In stock.,2021-10-28,12:03:11
536,"New Apple Watch SE (GPS, 44mm) - Space Grey Al...","27,900.00",In stock.,2021-10-28,12:03:23
537,"New Apple Watch SE (GPS, 44mm) - Space Grey Al...","27,900.00",In stock.,2021-10-28,12:03:34
538,"New Apple Watch SE (GPS, 44mm) - Space Grey Al...","27,900.00",In stock.,2021-10-28,12:03:46


*The following function will send us an "E-MAIL" if the price drops to the specified value.*

In [9]:
def e_mail():
    server = smtplib.SMTP_SSL('smtp.gmail.com',465)
    server.ehlo()
    server.ehlo()
    server.login('shuklarohan261@gmail.com','$$$$$$$$$$$$')
    subject = "Price Drop Alert!"
    body = "The smart watch is now available at your desired price.Hurry order it now! --> https://www.amazon.in/New-Apple-Watch-GPS-44mm/dp/B08J6HRBLH?ref_=Oct_DLandingS_D_8dc5a645_60&smid=A14CZOWI0VEHLG"
    msg = f"Subject: {subject}\n\n{body}"
    server.sendmail('shuklarohan261@gmail.com',msg)